In [1]:
import numpy as np
import pickle
import sklearn
from scipy import spatial
import pandas as pd

from sentence_transformers import util

from sklearn import metrics 
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

from matplotlib import pyplot as plt
import seaborn as sns

from dateutil.relativedelta import relativedelta

In [4]:
with open('D:\OneDrive - KU Leuven\KULeuven\Masterproef\data\\notebooks\\final_wa_df.pickle', 'rb') as handle:
    wa_df= pickle.load(handle)

with open('D:\OneDrive - KU Leuven\KULeuven\Masterproef\data\\notebooks\\final_stan_df.pickle', 'rb') as handle:
    stan_df = pickle.load(handle)

wa_content = wa_df.content
stan_content = stan_df.content

In [67]:
with open('classifierDf.pickle', 'rb') as handle:
    manualRatedDf = pickle.load(handle)

# with open('tfidfFitAllContent.pickle', 'rb') as handle:
#     vectorizer = pickle.load(handle)

# with open('LDAmodel.pickle', 'rb') as handle:
#     lda_model = pickle.load(handle)

# with open('allTfidfContentMatrix.pickle', 'rb') as handle:
#     tfidf_matrix = pickle.load(handle)

# with open('allLDAMatrix.pickle', 'rb') as handle:
#     lda_matrix = pickle.load(handle)

with open('allPreprocessedDf.pickle', 'rb') as handle:
    alldf = pickle.load(handle)

: 

In [6]:
alldf.head()

,filename,title,date,category,content,embeddings,content_no_stop,Newspaper,LDA,titleTFIDF
10112,wa936bik6.txt,Minder kinderen geboren,2013-01-08,België,Minder kinderen geboren In Vlaamse ziekenhuize...,"[0.8717681169509888, 0.21637214720249176, 0.67...",minder kinderen geboren vlaamse ziekenhuizen w...,Wablieft,"[0.8393840985830195, 0.017845665358110767, 0.0...","(0, 2478)\t0.10672176311979412\n (0, 3870)\..."
10113,wa936bik7.txt,Vlaams medicijn tegen tbc,2013-01-08,België,Vlaams medicijn tegen tbc Onderzoekers van het...,"[-0.1551680862903595, 0.6744857430458069, 0.13...",vlaams medicijn tbc onderzoekers vlaamse bedri...,Wablieft,"[0.8470347609801097, 0.01699313986956298, 0.01...","(0, 2478)\t0.10672176311979412\n (0, 3870)\..."
10114,wa936bik8.txt,Directrice verdacht van spieken,2013-01-08,België,Directrice verdacht van spieken De directrice ...,"[-0.3453265130519867, -0.008550132624804974, -...",directrice verdacht spieken directrice middelb...,Wablieft,"[0.7309227875702303, 0.017417658453388832, 0.0...","(0, 2478)\t0.10672176311979412\n (0, 3870)\..."
10118,wa936bin3.txt,Weinig vrouwen met sjerp in ons land,2013-01-08,België,Weinig vrouwen met sjerp in ons land In januar...,"[-0.2693834602832794, 0.9814454913139343, -0.1...",weinig vrouwen sjerp land januari beginnen nie...,Wablieft,"[0.689855816272154, 0.014164257376224907, 0.01...","(0, 2478)\t0.10672176311979412\n (0, 3870)\..."
10116,wa936bin1.txt,Noémie is Miss België,2013-01-08,België,Noémie is Miss België Noémie Happart is de moo...,"[-0.3678942322731018, -0.24943530559539795, -0...",noemie miss belgie noemie happart mooiste land...,Wablieft,"[0.5300806706135226, 0.014497165182422481, 0.0...","(0, 2478)\t0.10672176311979412\n (0, 3870)\..."


In [4]:
manualRatedDf

,WaTitle,WaContent,StanTitle,StanContent,Score,Match,tfidfTitle
0,Minder kinderen geboren,Minder kinderen geboren In Vlaamse ziekenhuize...,66.822,Geboortes 66.822 Het afgelopen jaar werden iet...,0.769700,1,0.000000
1,Vlaams medicijn tegen tbc,Vlaams medicijn tegen tbc Onderzoekers van het...,Vlaams medicijn brengt hoop voor stervenden me...,Ook Artsen zonder Grenzen vraagt al lang om he...,0.721415,1,0.822039
3,Directrice verdacht van spieken,Directrice verdacht van spieken De directrice ...,Directrice laat eigen kinderen examenvragen in...,Directrice laat eigen kinderen examenvragen in...,0.788607,1,0.375001
4,Noémie is Miss België,Noémie is Miss België Noémie Happart is de moo...,Noémie Happart Miss België 2013,Noémie Happart Miss België 2013 Noémie Happart...,0.836498,1,0.838483
5,Vlaams medicijn tegen tbc,Vlaams medicijn tegen tbc Onderzoekers van het...,Eerste nieuwsoortig tbc-medicijn in 40 jaar,Eerste nieuwsoortig tbc-medicijn in 40 jaar Ee...,0.704664,1,0.000000
...,...,...,...,...,...,...,...
129,Clara en Vincent achterna,Clara en Vincent achterna Actrice Clara Cleyma...,Nóg een oranje sofa,Buiten kun je onder meer dansen met een plasti...,0.556829,0,0.000000
130,Nog meer Harry Potter,Nog meer Harry Potter De Britse schrijfster J....,Drie nieuwe Harry Potter-films?,Film Drie nieuwe Harry Potter-films ? Filmprod...,0.741175,0,0.477470
133,Ruim het op!,Ruim het op! Vorige week zat ik in de klas van...,Warenhuisketen Colruyt bindt strijd aan met zw...,' Wanneer we de strijd willen aanbinden met zw...,0.576931,0,0.000000
134,China ontruimt kloosters,China ontruimt kloosters De overheid in China ...,Ook slachthuizen slachten liever niet onverdoofd,' De ene imam is nog strenger dan de andere en...,0.528937,0,0.000000


### Calculate LDA similarity

In [5]:
waTitleList = manualRatedDf["WaTitle"].tolist()
stanTitleList = manualRatedDf["StanTitle"].tolist()

In [30]:
LDAlist = []
for i in range(0, len(waTitleList)):
    waLDA = alldf.loc[alldf["title"] == waTitleList[i]]["LDA"].tolist()[0]
    stanLDA = alldf.loc[alldf["title"] == stanTitleList[i]]["LDA"].tolist()[0]
    LDAlist.append(1 - spatial.distance.cosine(waLDA, stanLDA))

In [35]:
manualRatedDf

,WaTitle,WaContent,StanTitle,StanContent,Score,Match,tfidfTitle,LDA
0,Minder kinderen geboren,Minder kinderen geboren In Vlaamse ziekenhuize...,66.822,Geboortes 66.822 Het afgelopen jaar werden iet...,0.769700,1,0.000000,1.000000
1,Vlaams medicijn tegen tbc,Vlaams medicijn tegen tbc Onderzoekers van het...,Vlaams medicijn brengt hoop voor stervenden me...,Ook Artsen zonder Grenzen vraagt al lang om he...,0.721415,1,0.822039,0.999487
3,Directrice verdacht van spieken,Directrice verdacht van spieken De directrice ...,Directrice laat eigen kinderen examenvragen in...,Directrice laat eigen kinderen examenvragen in...,0.788607,1,0.375001,0.991188
4,Noémie is Miss België,Noémie is Miss België Noémie Happart is de moo...,Noémie Happart Miss België 2013,Noémie Happart Miss België 2013 Noémie Happart...,0.836498,1,0.838483,0.941620
5,Vlaams medicijn tegen tbc,Vlaams medicijn tegen tbc Onderzoekers van het...,Eerste nieuwsoortig tbc-medicijn in 40 jaar,Eerste nieuwsoortig tbc-medicijn in 40 jaar Ee...,0.704664,1,0.000000,0.999884
...,...,...,...,...,...,...,...,...
129,Clara en Vincent achterna,Clara en Vincent achterna Actrice Clara Cleyma...,Nóg een oranje sofa,Buiten kun je onder meer dansen met een plasti...,0.556829,0,0.000000,0.998739
130,Nog meer Harry Potter,Nog meer Harry Potter De Britse schrijfster J....,Drie nieuwe Harry Potter-films?,Film Drie nieuwe Harry Potter-films ? Filmprod...,0.741175,0,0.477470,0.998270
133,Ruim het op!,Ruim het op! Vorige week zat ik in de klas van...,Warenhuisketen Colruyt bindt strijd aan met zw...,' Wanneer we de strijd willen aanbinden met zw...,0.576931,0,0.000000,0.993885
134,China ontruimt kloosters,China ontruimt kloosters De overheid in China ...,Ook slachthuizen slachten liever niet onverdoofd,' De ene imam is nog strenger dan de andere en...,0.528937,0,0.000000,0.958543


### Calculate most prevalent LDA topic

In [58]:
waLDA_list = []
stanLDA_list = []
for i in range(0, len(waTitleList)):
    waLDA = alldf.loc[alldf["title"] == waTitleList[i]]["LDA"].tolist()[0]
    stanLDA = alldf.loc[alldf["title"] == stanTitleList[i]]["LDA"].tolist()[0]
    waLDA_list.append(waLDA.index(max(waLDA))+1)
    stanLDA_list.append(stanLDA.index(max(stanLDA))+1)

In [62]:
manualRatedDf["WaLDA"] = waLDA_list
manualRatedDf["StanLDA"] = stanLDA_list

#### LDA topic match or not

In [64]:
match_list = []
for i in range(0, len(waTitleList)):
    waLDA = alldf.loc[alldf["title"] == waTitleList[i]]["LDA"].tolist()[0]
    stanLDA = alldf.loc[alldf["title"] == stanTitleList[i]]["LDA"].tolist()[0]
    if waLDA.index(max(waLDA)) == stanLDA.index(max(stanLDA)):
        match_list.append(1)
    else:
        match_list.append(0)

In [68]:
manualRatedDf["LDAMatch"] = match_list

In [76]:
manualRatedDf = manualRatedDf.drop('StanLDA', axis=1)

In [77]:
manualRatedDf

,WaTitle,WaContent,StanTitle,StanContent,Score,Match,tfidfTitle,DateDiff,LDAMatch
0,Minder kinderen geboren,Minder kinderen geboren In Vlaamse ziekenhuize...,66.822,Geboortes 66.822 Het afgelopen jaar werden iet...,0.769700,1,0.000000,5,1
1,Vlaams medicijn tegen tbc,Vlaams medicijn tegen tbc Onderzoekers van het...,Vlaams medicijn brengt hoop voor stervenden me...,Ook Artsen zonder Grenzen vraagt al lang om he...,0.721415,1,0.822039,6,1
3,Directrice verdacht van spieken,Directrice verdacht van spieken De directrice ...,Directrice laat eigen kinderen examenvragen in...,Directrice laat eigen kinderen examenvragen in...,0.788607,1,0.375001,5,1
4,Noémie is Miss België,Noémie is Miss België Noémie Happart is de moo...,Noémie Happart Miss België 2013,Noémie Happart Miss België 2013 Noémie Happart...,0.836498,1,0.838483,1,0
5,Vlaams medicijn tegen tbc,Vlaams medicijn tegen tbc Onderzoekers van het...,Eerste nieuwsoortig tbc-medicijn in 40 jaar,Eerste nieuwsoortig tbc-medicijn in 40 jaar Ee...,0.704664,1,0.000000,6,1
...,...,...,...,...,...,...,...,...,...
129,Clara en Vincent achterna,Clara en Vincent achterna Actrice Clara Cleyma...,Nóg een oranje sofa,Buiten kun je onder meer dansen met een plasti...,0.556829,0,0.000000,10,1
130,Nog meer Harry Potter,Nog meer Harry Potter De Britse schrijfster J....,Drie nieuwe Harry Potter-films?,Film Drie nieuwe Harry Potter-films ? Filmprod...,0.741175,0,0.477470,13,1
133,Ruim het op!,Ruim het op! Vorige week zat ik in de klas van...,Warenhuisketen Colruyt bindt strijd aan met zw...,' Wanneer we de strijd willen aanbinden met zw...,0.576931,0,0.000000,0,1
134,China ontruimt kloosters,China ontruimt kloosters De overheid in China ...,Ook slachthuizen slachten liever niet onverdoofd,' De ene imam is nog strenger dan de andere en...,0.528937,0,0.000000,13,1


### Calculate publication diff

In [38]:
dateDiff = []
for i in range(0, len(waTitleList)):
    waDate = alldf.loc[alldf["title"] == waTitleList[i]]["date"].tolist()[0]
    stanDate = alldf.loc[alldf["title"] == stanTitleList[i]]["date"].tolist()[0]
    dateDiff.append((waDate - stanDate).days)

In [47]:
manualRatedDf["DateDiff"] = dateDiff

In [50]:
manualRatedDf

,WaTitle,WaContent,StanTitle,StanContent,Score,Match,tfidfTitle,LDA,DateDiff
0,Minder kinderen geboren,Minder kinderen geboren In Vlaamse ziekenhuize...,66.822,Geboortes 66.822 Het afgelopen jaar werden iet...,0.769700,1,0.000000,1.000000,5
1,Vlaams medicijn tegen tbc,Vlaams medicijn tegen tbc Onderzoekers van het...,Vlaams medicijn brengt hoop voor stervenden me...,Ook Artsen zonder Grenzen vraagt al lang om he...,0.721415,1,0.822039,0.999487,6
3,Directrice verdacht van spieken,Directrice verdacht van spieken De directrice ...,Directrice laat eigen kinderen examenvragen in...,Directrice laat eigen kinderen examenvragen in...,0.788607,1,0.375001,0.991188,5
4,Noémie is Miss België,Noémie is Miss België Noémie Happart is de moo...,Noémie Happart Miss België 2013,Noémie Happart Miss België 2013 Noémie Happart...,0.836498,1,0.838483,0.941620,1
5,Vlaams medicijn tegen tbc,Vlaams medicijn tegen tbc Onderzoekers van het...,Eerste nieuwsoortig tbc-medicijn in 40 jaar,Eerste nieuwsoortig tbc-medicijn in 40 jaar Ee...,0.704664,1,0.000000,0.999884,6
...,...,...,...,...,...,...,...,...,...
129,Clara en Vincent achterna,Clara en Vincent achterna Actrice Clara Cleyma...,Nóg een oranje sofa,Buiten kun je onder meer dansen met een plasti...,0.556829,0,0.000000,0.998739,10
130,Nog meer Harry Potter,Nog meer Harry Potter De Britse schrijfster J....,Drie nieuwe Harry Potter-films?,Film Drie nieuwe Harry Potter-films ? Filmprod...,0.741175,0,0.477470,0.998270,13
133,Ruim het op!,Ruim het op! Vorige week zat ik in de klas van...,Warenhuisketen Colruyt bindt strijd aan met zw...,' Wanneer we de strijd willen aanbinden met zw...,0.576931,0,0.000000,0.993885,0
134,China ontruimt kloosters,China ontruimt kloosters De overheid in China ...,Ook slachthuizen slachten liever niet onverdoofd,' De ene imam is nog strenger dan de andere en...,0.528937,0,0.000000,0.958543,13


In [78]:
with open('classifierDF.pickle', 'wb') as handle:
    pickle.dump(manualRatedDf, handle, protocol=pickle.HIGHEST_PROTOCOL)

### creation of df previously evaluated articles

In [2]:
df = pd.read_csv('D:\OneDrive - KU Leuven\KULeuven\Masterproef\data\ManualScoringLog.csv', delimiter=', ')
df.top_3_standaard_idx = df.top_3_standaard_idx.apply(lambda x: x.strip("[]").replace("'","").split(","))
df.Manual_Scores = df.Manual_Scores.apply(lambda x: x.strip("[]").replace("'","").split(","))

WablieftIndexList = df['Wablieft_idx'].tolist()
StandaardIndexList = df['top_3_standaard_idx'].tolist()
SimScoreList = df['Manual_Scores'].tolist()

with open('D:\OneDrive - KU Leuven\KULeuven\Masterproef\data\wa_limited_testdf.pickle', 'rb') as handle:
    limited_wa_df = pickle.load(handle)

with open('D:\OneDrive - KU Leuven\KULeuven\Masterproef\data\stan_limited_testdf.pickle', 'rb') as handle:
    limited_stan_df = pickle.load(handle)

wa_embeddings = np.load('D:\OneDrive - KU Leuven\KULeuven\Masterproef\data\\bert\wa_testbatch_embeddings.npy')
stan_embeddings = np.load('D:\OneDrive - KU Leuven\KULeuven\Masterproef\data\\bert\stan_testbatch_embeddings.npy')
    

c:\Users\Nick\anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [26]:
SimScoreList = [[int(number) for number in sub] for sub in SimScoreList]

In [66]:
manualRatedDf

,WaTitle,WaContent,StanTitle,StanContent,Score,Match,tfidfTitle,DateDiff,LDAMatch
0,Minder kinderen geboren,Minder kinderen geboren In Vlaamse ziekenhuize...,66.822,Geboortes 66.822 Het afgelopen jaar werden iet...,0.769700,1,0.000000,5,1
1,Vlaams medicijn tegen tbc,Vlaams medicijn tegen tbc Onderzoekers van het...,Vlaams medicijn brengt hoop voor stervenden me...,Ook Artsen zonder Grenzen vraagt al lang om he...,0.721415,1,0.822039,6,1
3,Directrice verdacht van spieken,Directrice verdacht van spieken De directrice ...,Directrice laat eigen kinderen examenvragen in...,Directrice laat eigen kinderen examenvragen in...,0.788607,1,0.375001,5,1
4,Noémie is Miss België,Noémie is Miss België Noémie Happart is de moo...,Noémie Happart Miss België 2013,Noémie Happart Miss België 2013 Noémie Happart...,0.836498,1,0.838483,1,0
5,Vlaams medicijn tegen tbc,Vlaams medicijn tegen tbc Onderzoekers van het...,Eerste nieuwsoortig tbc-medicijn in 40 jaar,Eerste nieuwsoortig tbc-medicijn in 40 jaar Ee...,0.704664,1,0.000000,6,1
...,...,...,...,...,...,...,...,...,...
129,Clara en Vincent achterna,Clara en Vincent achterna Actrice Clara Cleyma...,Nóg een oranje sofa,Buiten kun je onder meer dansen met een plasti...,0.556829,0,0.000000,10,1
130,Nog meer Harry Potter,Nog meer Harry Potter De Britse schrijfster J....,Drie nieuwe Harry Potter-films?,Film Drie nieuwe Harry Potter-films ? Filmprod...,0.741175,0,0.477470,13,1
133,Ruim het op!,Ruim het op! Vorige week zat ik in de klas van...,Warenhuisketen Colruyt bindt strijd aan met zw...,' Wanneer we de strijd willen aanbinden met zw...,0.576931,0,0.000000,0,1
134,China ontruimt kloosters,China ontruimt kloosters De overheid in China ...,Ook slachthuizen slachten liever niet onverdoofd,' De ene imam is nog strenger dan de andere en...,0.528937,0,0.000000,13,1


In [40]:
wa_filelist = []
wa_titlelist = []
wa_contentlist = []
stan_filelist = []
stan_titlelist = []
stan_contentlist = []
sim_list = []
score_list = []
for i in range(0, len(WablieftIndexList)):
    waContent = limited_wa_df.iloc[int(WablieftIndexList[i])].content
    wa_embedding = wa_df.loc[wa_df["content"] == waContent].embeddings.tolist()[0]
    wa_filelist.append(wa_df.loc[wa_df["content"] == waContent].filename.tolist()[0])
    idx = SimScoreList[i].index(max(SimScoreList[i]))
    stanContent = limited_stan_df.iloc[int(StandaardIndexList[i][idx])].content
    stan_embedding = stan_df.loc[stan_df["content"] == stanContent].embeddings.tolist()[0]
    stan_filelist.append(stan_df.loc[stan_df["content"] == stanContent].filename)
    sim_list.append(util.cos_sim(wa_embedding, stan_embedding).tolist()[0][0])
    if (float(SimScoreList[i][idx]) / 5) == 1:
        score_list.append(1)
    else:
        score_list.append(0)


In [42]:
firstEvalDf = pd.DataFrame()
firstEvalDf["WaFile"] = wa_filelist
firstEvalDf["StanFile"] = stan_filelist
firstEvalDf["Score"] = sim_list
firstEvalDf["Match"] = score_list

In [63]:
firstEvalDf

,WaFile,StanFile,Score,Match
0,wa1024mening.txt,15473 20141106_319f7214-6508-11e4-a276-9dab...,0.605905,0
1,wa1023vp.txt,15470 20141106_2f85cab6-6501-11e4-a276-9dab...,0.755112,0
2,wa1017mening.txt,11295 20141003_c03dfeac-4a67-11e4-bb48-738d...,0.665797,0
3,wa1021spo3.txt,13466 20141021_3d52dc42-588b-11e4-a11a-c578...,0.831735,1
4,wa1019we5.txt,11264 20141003_8efd3cce-4a47-11e4-bb48-738d...,0.566712,0
...,...,...,...,...
538,wa1015tip1.txt,16105 20141112_307ad4b0-699e-11e4-804c-2d41...,0.590103,0
539,wa1017bik4.txt,16923 20141118_e48023f6-6e95-11e4-acb5-b808...,0.707330,0
540,wa1018idk3.txt,11440 20141004_bac2604e-4acb-11e4-aaf6-a739...,0.668812,0
541,wa1026we3.txt,13393 20141020_c1aa1a64-55f2-11e4-92ed-c7b6...,0.546066,0
